# By using CuPy, reimplement bifurcation map (Task 1).

In [4]:
pip install cupy

In [38]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import cProfile

In [8]:
def bif_map(r,x):
    return r*x*(1-x)

In [13]:
n_iter = 1000
n_r = 100_000

## Numpy

In [36]:
def bif_map_numpy(n_r=n_r,n_iter=n_iter):
    R_np = np.linspace(0,4,n_r)
    X_np = np.random.rand(len(R_np))
    for i in range(n_iter):
        X_np = bif_map(R_np,X_np)
    return R_np, X_np

In [37]:
plt.figure(figsize=(14,8))
R_np, X_np = bif_map_numpy()
plt.scatter(R_np, X_np,c='black',s=0.1)
plt.xlim(0,4)
plt.xlabel("$r$",fontsize=30)
plt.ylabel("$x_\infty$",fontsize=30)
plt.title('Bifurcation map, Numpy, $x \in [0,4]$',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.show()

## Cupy

In [32]:
def bif_map_cupy(n_r=n_r,n_iter=n_iter):
    R_cp = cp.linspace(0,4,n_r)
    X_cp = cp.random.rand(len(R_np))
    for i in range(n_iter):
        X_cp = bif_map(R_cp,X_cp)
    return R_cp.get(), X_cp.get()

In [34]:
R_cp, X_cp = bif_map_cupy()
plt.figure(figsize=(14,8))
plt.scatter(R_cp, X_cp,c='black',s=0.1)
plt.xlim(0,4)
plt.xlabel("$r$",fontsize=30)
plt.ylabel("$x_\infty$",fontsize=30)
plt.title('Bifurcation map, Cupy, $x \in [0,4]$',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.show()

## Profile

In [39]:
cProfile.run('bif_map_cupy(10_000,100_000)')

MPI

In [2]:
!apt -y -q install libopenmpi-dev

In [3]:
pip install mpi4py

In [4]:
from mpi4py import MPI
import time

In [5]:
%%writefile parallel_bifurcation.py

from mpi4py import MPI
import numpy as np
import sys

n_r = int(sys.argv[1])
n_iter = int(sys.argv[2])

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

R_total = np.linspace(0,4,n_r)
R = np.array_split(R_total, size)[rank]


X = np.random.rand(len(R))
for i in range(n_iter): X=(R*X)*(1-X)

X_total_temp = comm.gather(X, root = 0)
if rank == 0:
    X_total = np.concatenate(X_total_temp)

In [6]:
T = []
for i in range(1,11):
    t_start = time.time()
    !mpiexec -n {i} --allow-run-as-root python  parallel_bifurcation.py 500000 10000
    t_end = time.time()
    T.append(t_end-t_start)
    print('Time: {:6.3f}     Processes: {}'.format(T[-1],i))

## Graph